In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
H = np.loadtxt("Hmatrix2.txt",delimiter=',')
Axes0 = H.shape[0] #Rows
Axes1 = H.shape[1] #Cols
print(Axes0,Axes1)
# print(H)

In [ ]:
class Node:
    def __init__(self, nodeID, connected_nodeList, bitValue,*args,**kwargs): 
        self.nodeID = nodeID #Helps to locate node in general list
        self.adjNodes = connected_nodeList # List of connected nodes to a particular node
        self.prevMsg = 2 #Set 2 as there is no initial messsage when started
        self.currMsg = bitValue
        self.FrmAdjNode = [None]*len(connected_nodeList) #Msg received from adj nodes


    def varToChk(self,chkNodes):
        if self.prevMsg==2:
            # print("First Iteration\n")
            for nodeInd in self.adjNodes: # From variable nodes adjacent node list 
                idInd = chkNodes[nodeInd].adjNodes.index(self.nodeID) #Index of checknodes in chkNodes list
                chkNodes[nodeInd].FrmAdjNode[idInd] = self.currMsg
                # print("ChkNodeID: ",chkNodes[nodeInd].nodeID, chkNodes[nodeInd].adjNodes," ProbabList: ",chkNodes[nodeInd].FrmAdjNode)
            self.prevMsg = self.currMsg
        else:
            # print("2+ Iteration\n")
            self.prevMsg = self.currMsg
            for i in range(len(self.adjNodes)):
                flag=0
                # print(f"\npr1:{pr1} ProbabList: ",self.FrmAdjNode)
                for j in range(len(self.FrmAdjNode)): #Iterate through probab list sent by chk nodes
                    if  i == j:
                        continue
                    else:
                        if self.FrmAdjNode[j] !=-1:
                            msgToChk = self.FrmAdjNode[j] #Since VN's a repetation code
                            flag=1
                            break
                        # print(prob1)
                if flag==0 and self.currMsg==-1:
                    msgToChk = -1
                else:
                    msgToChk = self.currMsg
                chkNodeInd = self.adjNodes[i]
                varNodeInd = chkNodes[chkNodeInd].adjNodes.index(self.nodeID)
                chkNodes[chkNodeInd].FrmAdjNode[varNodeInd] = msgToChk
                # print("ChkNodeID: ",chkNodes[chkNodeInd].nodeID, chkNodes[chkNodeInd].adjNodes,f" k: {k} ")

            
            # for bit in self.FrmAdjNode:
            if self.currMsg == -1:
                if 0 in self.FrmAdjNode:
                    self.currMsg = 0
                elif 1 in self.FrmAdjNode:
                    self.currMsg = 1
                

    def chkToVar(self,varNodes):
        for i in range(len(self.adjNodes)):
            parity = 0
            for j in range(len(self.FrmAdjNode)): #Iterate through probab list sent by var nodes
                rec_bit = self.FrmAdjNode[j]
                if i == j:
                    continue
                elif rec_bit == -1:
                    parity=-1
                    break
                else:
                    parity = parity^rec_bit
                    
            varNodeInd = self.adjNodes[i]
            chknodeInd = varNodes[varNodeInd].adjNodes.index(self.nodeID)
            varNodes[varNodeInd].FrmAdjNode[chknodeInd] =  parity
            # print("VarNodeID: ",varNodes[varNodeInd].nodeID, varNodes[varNodeInd].adjNodes," ProbabList: ",varNodes[varNodeInd].FrmAdjNode)


In [ ]:
def hard_decode_bec(varNodes,chkNodes):
    t = 0 #Iteration index

    while(t<25):
        # print(f"Iteration {t}")
        #variable node to check node logic
        for varNode in varNodes:
            # print(f"Var{varNode.nodeID} with bit prob:{varNode.pr1} to chk\n")
            varNode.varToChk(chkNodes)

        #check node to variable node logic
        flag = [1]*len(chkNodes)
        for i in range(len(chkNodes)):
            chkNode = chkNodes[i]
            # print(f"chk{chkNode.nodeID} to Var\n")
            flag[i] = chkNode.chkToVar(varNodes)
        
                
        #Loop break condition and error counting
        currWord = []
        errCnt = 0
        for parity in varNodes:
            currWord.append(parity.currMsg)
        

        # if 1 not in flag:
        #     return errorList,currWord
        # if -1 not in currWord:
        #     # print(currWord)
        #     if sum(currWord)%2 == 0:
        #         return errorList,currWord

            
        t += 1
    return currWord

In [ ]:
def generate_msg(p):
    # p = 0.3 #Probability of a bit to be an erasure
    Msg = np.random.randint(0, 1, size=Axes1)
    receivedMsg = Msg.copy()
    receivedMsg[np.random.rand(Axes1) < p] = -1
    # print(receivedMsg)
    return list(Msg),list(receivedMsg)

In [ ]:
def initNodes(receivedMsg):
    varNodes = []
    chkNodes = []
    for i in range(Axes0): 
        temp1 = list(H[i].nonzero()[0]) #Finding adj nodes
        chkNodes.append(Node(nodeID=i, connected_nodeList=temp1, bitValue=2)) #Checknode initially has no message

    for i in range(Axes1): 
        temp2 = list(H[:,i].nonzero()[0])
        varNodes.append(Node(nodeID=i, connected_nodeList=temp2, bitValue=receivedMsg[i])) #Initializing variable nodes with received message
    
    return varNodes,chkNodes


In [ ]:
def simulate_bec_monte_carlo_parallel(p, Nsim=100):

    F = np.zeros(Nsim) #Flag
    
    for l in range(Nsim):
        x,r = generate_msg(p)
        # Decode received signal
        varNodes,chkNodes = initNodes(r)
        x_dec = hard_decode_bec(varNodes,chkNodes)

        # Check if decoding was successful
        if np.all(x_dec == x):
            F[l] = 1

    # with concurrent.futures.ThreadPoolExecutor() as executor:
    #     futures = [executor.submit(simulate, l) for l in range(Nsim)]
    
    psd = np.mean(F) #probability of successful decoding
    # global p_psd
    # p_psd[p] = psd
    return psd

#Usage:
p_values = np.arange(0, 1.00, 0.08)
# p_values = np.array([0.12,0.25,0.38,0.41,0.55,0.69,0.72,0.82,0.92,0.99,1.0])
pe_values = [simulate_bec_monte_carlo_parallel(p) for p in p_values] #Prob of error
plt.plot(p_values, pe_values)
plt.xlabel("Probability of erasure (p)")
plt.ylabel("Probability of successful decoding")
plt.title("Performance of LDPC decoding over a BEC")
plt.show()